
<div style="background-color:#CCCCCC; padding:12px; border-radius:8px;">
<h1 style="color:#003366; text-align:center; margin:8px 0;">Revisión y limpieza de 3 DataFrames (TPI - Data Analytics)</h1>
<p style="text-align:center; color:#003366; margin:0;"><em>Notebook docente en castellano — nombres descriptivos en snake_case — código y documentación</em></p>
</div>



<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center; margin-top:6px;">Resumen</h2>

<p style="color:black;">
Este notebook está diseñado con finalidades pedagógicas. Revisa, normaliza y valida tres datasets contenidos en CSV:
</p>

<ul style="color:black;">
<li><code>marketing.csv</code> → variable: <code>df_marketing</code></li>
<li><code>ventas.csv</code> → variable: <code>df_ventas</code></li>
<li><code>clientes.csv</code> → variable: <code>df_clientes</code></li>
</ul>

<p style="color:black;">
Coloca los CSV en <code>./data_in/</code> o en <code>/mnt/data/</code>. El notebook busca primero en <code>./data_in/</code> y si no encuentra, usa <code>/mnt/data/</code> (útil para entornos donde los archivos están pre-subidos).
</p>

</div>


In [19]:
# Imports y configuración inicial (nombres en castellano)
import os
from pathlib import Path
import json
import unicodedata
import zipfile

#!pip install gdown

import argparse
import pandas as pd
import numpy as np

import zipfile
from collections import defaultdict
from datetime import datetime
from math import isnan
ruta_base = ""


## 1. Crear un documento en Google Colaboratory y cargar los sets de datos como DataFrames

In [20]:
# Rutas: busca en ./data_in primero, si no existe usa /mnt/data

carpeta_entrada    = Path(ruta_base)
#carpeta_entrada_mnt   = Path('/mnt/data')

carpeta_reportes   = carpeta_entrada / 'reportes'
carpeta_reportes.mkdir(parents=True, exist_ok=True)

carpeta_limpios    = carpeta_entrada / 'limpios'
carpeta_limpios.mkdir(parents=True, exist_ok=True)

# Nombres esperados de archivos
archivo_ventas     = 'ventas.csv'
archivo_clientes   = 'clientes.csv'
archivo_marketing  = 'marketing.csv'

In [21]:
# --- Paso 3: Cargar archivos del curso ---
print("Cargando datasets del curso...")
ruta_ventas     = os.path.join(ruta_base, archivo_ventas)
ruta_clientes   = os.path.join(ruta_base, archivo_clientes)
ruta_marketing  = os.path.join(ruta_base, archivo_marketing)

df_ventas    = pd.read_csv(f"{ruta_ventas}")
df_clientes  = pd.read_csv(f"{ruta_clientes}")
df_marketing = pd.read_csv(f"{ruta_marketing}")

Cargando datasets del curso...


In [22]:
# Mostrar el DataFrame
print(f"""
df_ventas
{df_ventas}
{"*"*50}
df_clientes
{df_clientes}
{"*"*50}
df_marketing
{df_marketing}""")


df_ventas
      id_venta                producto   precio  cantidad fecha_venta  \
0          792       Cuadro decorativo   $69.94       5.0  02/01/2024   
1          811         Lámpara de mesa  $105.10       5.0  02/01/2024   
2         1156                Secadora   $97.96       3.0  02/01/2024   
3         1372                Heladera  $114.35       8.0  02/01/2024   
4         1546                Secadora  $106.21       4.0  02/01/2024   
...        ...                     ...      ...       ...         ...   
3030      1837         Horno eléctrico  $104.12       9.0  30/12/2024   
3031      2276                  Laptop   $85.27       9.0  30/12/2024   
3032      2696                  Laptop  $107.81       4.0  30/12/2024   
3033      2913              Smartphone   $99.85       7.0  30/12/2024   
3034      2930  Consola de videojuegos   $55.47       6.0  30/12/2024   

              categoria  
0            Decoración  
1            Decoración  
2     Electrodomésticos  
3     El

In [23]:
# ---------- Funciones utilitarias en castellano (snake_case) ----------
def sacar_acentos(texto):
    """
    Elimina acentos (tildes/diacríticos) de un texto.
    Mantiene NaN intactos.
    """
    if pd.isna(texto):
        return texto
    texto = str(texto)
    nk = unicodedata.normalize('NFKD', texto)
    return ''.join([c for c in nk if not unicodedata.combining(c)])

In [24]:
TOKENS_VALOR_FALTANTE = {
    '', 'na', 'n/a', 'null', 'none', 'sin dato', 's/d', 'nd', '-', '--', '?', 'sin_dato', 'n/d'
}

def es_valor_faltante(valor):
    """
    Determina si un valor debe considerarse faltante (True) usando tokens y NaN.
    """
    if pd.isna(valor):
        return True
    s = str(valor).strip().lower()
    s = sacar_acentos(s)
    return s in TOKENS_VALOR_FALTANTE

In [25]:
# ---------- Funciones de limpieza ----------
def aplicar_regla_columna(serie, regla):
    """
    Aplica una regla a una serie (columna).
    Firma compatible con la versión anterior (reemplaza la implementación previa).
    regla puede ser:
      - 'strip','lower','upper','title','quitar_acentos','numeric','date'
    o un tuple (tipo, opciones) con opciones:
      - numeric: remove_non_digits (bool), remove_thousands (bool), as_int (bool), thousands_separator (',' o '.')
      - date: formats (list), dayfirst (bool), format_output ('YYYY/MM/DD' para forzar cadena)
      - texto: normalizar_acentos (bool)
    Retorna la serie transformada (sin forzar dtype final).
    """
    tipo, opts = regla if isinstance(regla, tuple) else (regla, {})
    opts = opts or {}
    s = serie.copy()

    # -------- Texto y normalización de acentos opcional --------
    if tipo == 'strip':
        s = s.map(lambda x: str(x).strip() if not pd.isna(x) else x)
    elif tipo == 'lower':
        s = s.map(lambda x: str(x).strip().lower() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'upper':
        s = s.map(lambda x: str(x).strip().upper() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'title':
        s = s.map(lambda x: str(x).strip().title() if not pd.isna(x) else x)
        if opts.get('normalizar_acentos'):
            s = s.map(lambda x: sacar_acentos(x) if not pd.isna(x) else x)
    elif tipo == 'quitar_acentos':
        s = s.map(lambda x: sacar_acentos(x).strip() if not pd.isna(x) else x)

    # -------- Numeric robusto --------
    elif tipo == 'numeric':
        def to_num(v):
            if pd.isna(v):
                return np.nan
            t = str(v).strip()
            t = t.replace('$', '')
            if opts.get('remove_non_digits', False):
                t = ''.join([c for c in t if c.isdigit() or c in '.-'])
            if opts.get('remove_thousands', False):
                sep = opts.get('thousands_separator', ',')
                if sep == ',':
                    # Quitar puntos mil y comas de decimales no soportadas -> suponer coma miles y punto decimales
                    t = t.replace('.', '').replace(',', '')
                else:
                    t = t.replace(',', '').replace('.', '')   
            try:
                val = pd.to_numeric(t, errors='coerce')
                if opts.get('as_int', False):
                    if pd.isna(val):
                        return pd.NA
                    try:
                        # intentar entero simple
                        return int(val)
                    except Exception:
                        return pd.NA
                return float(val) if not pd.isna(val) else np.nan
            except Exception:
                return np.nan
        s = s.map(to_num)

    # -------- Fecha robusta --------
    elif tipo == 'date':
        formatos = opts.get('formats', [])
        dayfirst = opts.get('dayfirst', True)
        def to_date(v):
            if pd.isna(v):
                return pd.NaT
            t = str(v).strip()
            # Probar formatos explícitos
            for fmt in formatos:
                try:
                    return pd.to_datetime(datetime.strptime(t, fmt))
                except Exception:
                    continue
            # Fallback: pandas con dayfirst
            try:
                return pd.to_datetime(t, dayfirst=dayfirst, errors='coerce')
            except Exception:
                return pd.NaT
        s = s.map(to_date)

    else:
        # Default: trim y convertir tokens faltantes a NaN para texto
        if s.dtype == object or pd.api.types.is_string_dtype(s):
            s = s.map(lambda x: np.nan if es_valor_faltante(x) else (str(x).strip() if not pd.isna(x) else x))
    return s

In [26]:
def convertir_tipos_postprocesamiento(df, reglas_por_columna):
    """
    Garantiza dtypes correctos:
     - Para columnas numeric: pd.to_numeric(...) + conversión a Int64 nullable si as_int, o float.
     - Para columnas date: intenta parsear según formatos; deja datetime64[ns] o, si 'format_output'=='YYYY/MM/DD', devuelve strings con ese formato.
    """
    df2 = df.copy()
    for col, regla in (reglas_por_columna or {}).items():
        tipo = regla if not isinstance(regla, tuple) else regla[0]
        opts = {} if not isinstance(regla, tuple) else regla[1] or {}
        if tipo == 'numeric':
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
            if opts.get('as_int', False):
                # convertir a Int64 nullable
                try:
                    df2[col] = df2[col].astype('Int64')
                except Exception:
                    # fallback: mantener float si conversion falla
                    df2[col] = pd.to_numeric(df2[col], errors='coerce')
        elif tipo == 'date':
            formatos = opts.get('formats', [])
            dayfirst = opts.get('dayfirst', True)
            parsed = pd.Series(pd.NaT, index=df2.index)
            # probar formatos explícitos uno por uno
            for fmt in formatos:
                try:
                    mask_necesita = parsed.isna()
                    intent = pd.to_datetime(df2.loc[mask_necesita, col].astype(str), format=fmt, errors='coerce')
                    parsed.loc[mask_necesita] = intent
                except Exception:
                    pass
            # fallback general para los que quedaron NaT
            still_na = parsed.isna()
            if still_na.any():
                parsed.loc[still_na] = pd.to_datetime(df2.loc[still_na, col].astype(str), dayfirst=dayfirst, errors='coerce')
            df2[col] = parsed
            if opts.get('format_output') == 'YYYY/MM/DD':
                # convertir a string con formato pedido (mantener NaT como NaN)
                df2[col] = df2[col].dt.strftime('%Y/%m/%d')
    for col in df2.select_dtypes(include=['object']).columns:
        df2[col] = df2[col].map(sacar_acentos)
    return df2

In [27]:
def limpiar_dataframe(df, reglas_por_columna=None):
    """
    Limpieza principal (misma firma que antes):
     1) Aplica aplicar_regla_columna por cada columna según reglas_por_columna
     2) Para columnas texto por defecto: strip + tokens faltantes -> NaN
     3) Elimina duplicados exactos
     4) Convierte tipos numéricos y fechas con convertir_tipos_postprocesamiento
    Retorna df limpio con dtypes corregidos.

    Aplica reglas de limpieza columna por columna.
    Si una columna no existe en el DataFrame, se omite con aviso educativo.
    """
    #reglas_por_columna = reglas_por_columna or {}

    reglas_por_columna = reglas_por_columna or {}
    df2 = df.copy()
    df2.columns = [str(c).strip() for c in df2.columns]

    for col in df2.columns:
        if col in reglas_por_columna:
            # usar la función aplicar_regla_columna existente en el notebook
            df2[col] = aplicar_regla_columna(df2[col], reglas_por_columna[col])
        else:
            if df2[col].dtype == object or pd.api.types.is_string_dtype(df2[col]):
                df2[col] = df2[col].map(lambda x: np.nan if es_valor_faltante(x) else (str(x).strip() if not pd.isna(x) else x))

    # eliminar duplicados exactos
    df2 = df2.drop_duplicates(keep='first').reset_index(drop=True)

    # intentos de post-conversión sencillos para numeric/date según reglas:
    for col, regla in (reglas_por_columna or {}).items():
        tipo = regla if not isinstance(regla, tuple) else regla[0]
        opts = {} if not isinstance(regla, tuple) else regla[1] or {}
        if tipo == 'numeric':
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
            if opts.get('as_int', False):
                try:
                    df2[col] = df2[col].astype('Int64')
                except Exception:
                    pass
        elif tipo == 'date':
            formatos = opts.get('formats', [])
            dayfirst = opts.get('dayfirst', True)
            parsed = pd.Series(pd.NaT, index=df2.index)
            for fmt in formatos:
                try:
                    mask = parsed.isna()
                    parsed.loc[mask] = pd.to_datetime(df2.loc[mask, col].astype(str), format=fmt, errors='coerce')
                except Exception:
                    pass
            still_na = parsed.isna()
            if still_na.any():
                parsed.loc[still_na] = pd.to_datetime(df2.loc[still_na, col].astype(str), dayfirst=dayfirst, errors='coerce')
            # si se pidió format_output, devolver cadena con YYYY/MM/DD
            if opts.get('format_output') == 'YYYY/MM/DD':
                df2[col] = parsed.dt.strftime('%Y/%m/%d')
            else:
                df2[col] = parsed

    return df2

In [28]:
# Detección de outliers (IQR)
def mascara_valores_atipicos_rango_intercuartil(serie_datos):
    """
    Devuelve máscara booleana (True = outlier) según IQR.
    """
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    q1 = serie_limpia.quantile(0.25)
    q3 = serie_limpia.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr    
    return (serie_datos < limite_inferior) | (serie_datos > limite_superior),int(out_iqr.sum()),(float(lb), float(ub))

In [29]:
# Detección de outliers (Z-score)
def mascara_valores_atipicos_zscore(serie_datos, umbral=3.0):
    """
    Devuelve máscara booleana (True = outlier) según Z-score.
    """
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4 or serie_limpia.std() == 0:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    puntaje_z = (serie_datos - serie_limpia.mean()) / serie_limpia.std()
    return puntaje_z.abs() > umbral    

In [30]:
print('Funciones utilitarias definidas.')

Funciones utilitarias definidas.


# limpieza y normalización

In [31]:
# ---------- Detección de problemas en un DataFrame ----------
def detectar_problemas_en_dataframe(df: pd.DataFrame):
    """Detecta problemas comunes en un DataFrame y retorna:
    - resumen: diccionario con métricas generales
    - chequeos_por_columna: dict con información por columna
    - problemas_df: DataFrame con filas problemáticas y descripción
    """
    resumen = {}
    resumen['filas'] = df.shape[0]
    resumen['columnas'] = df.shape[1]
    resumen['nulos_por_columna'] = df.isna().sum().to_dict()
    dup_mask = df.duplicated(keep=False)
    resumen['duplicados_exactos'] = int(dup_mask.sum())

    chequeos_por_columna = {}
    for col in df.columns:
        ser = df[col]
        info = {'dtype': str(ser.dtype), 'nulos': int(ser.isna().sum())}
        # texto
        if ser.dtype == object or pd.api.types.is_string_dtype(ser):
            s = ser.astype(str)
            info['espacios_inicio'] = int(s.str.match(r'^\s+').sum())
            info['espacios_final'] = int(s.str.match(r'\s+$').sum())
            try:
                unique_original = set(s.dropna().unique())
                unique_lower = set(s.dropna().str.lower().unique())
                info['unique_original'] = len(unique_original)
                info['unique_lower'] = len(unique_lower)
                info['variantes_mayusculas'] = len(unique_lower) < len(unique_original)
            except Exception:
                info['unique_original'] = ser.nunique(dropna=True)
                info['unique_lower'] = None
                info['variantes_mayusculas'] = None
            try:
                unaccented = s.dropna().map(lambda x: sacar_acentos(x).lower())
                groups = unaccented.groupby(unaccented).size()
                conflicts = groups[groups > 1]
                info['grupos_variantes_acentos'] = int(conflicts.shape[0])
                ejemplos = {}
                if not conflicts.empty:
                    for val in conflicts.index[:5]:
                        originales = sorted(list(s[unaccented == val].unique())[:10])
                        ejemplos[val] = originales
                info['ejemplos_variantes_acentos'] = ejemplos
            except Exception:
                info['grupos_variantes_acentos'] = None
                info['ejemplos_variantes_acentos'] = {}
            info['tokens_aparente_faltante'] = int(
                s.map(lambda x: str(x).strip().lower()).map(lambda v: sacar_acentos(v) in TOKENS_VALOR_FALTANTE).sum()
            )
            info['muestras'] = list(s.dropna().unique()[:10])
        else:
            # numeric
            if pd.api.types.is_numeric_dtype(ser) or (ser.dropna().astype(str).str.replace('.','',1).str.isnumeric().all() if len(ser.dropna())>0 else False):
                try:
                    s_f = ser.dropna().astype(float)
                except Exception:
                    s_f = pd.to_numeric(ser, errors='coerce').dropna().astype(float)
                info['media'] = float(s_f.mean()) if not s_f.empty else None
                info['std'] = float(s_f.std()) if not s_f.empty else None
                info['min'] = float(s_f.min()) if not s_f.empty else None
                info['max'] = float(s_f.max()) if not s_f.empty else None
               
                if len(s_f) >= 4:
                    q1 = s_f.quantile(0.25)
                    q3 = s_f.quantile(0.75)
                    iqr = q3 - q1
                    lb = q1 - 1.5 * iqr
                    ub = q3 + 1.5 * iqr
                    out_iqr,info['outliers_iqr'] ,info['limites_iqr'] =mascara_valores_atipicos_rango_intercuartil(col)
                    """
                    out_iqr = (s_f < lb) | (s_f > ub)
                    info['outliers_iqr'] = int(out_iqr.sum())
                    info['limites_iqr'] = (float(lb), float(ub))"""
                else:
                    info['outliers_iqr'] = None
                    info['limites_iqr'] = None
                if len(s_f) >= 4 and s_f.std() != 0:
                    z = (s_f - s_f.mean()) / s_f.std()
                    info['outliers_z'] = int((z.abs() > 3).sum())
                else:
                    info['outliers_z'] = None
            else:
                # fechas intento parseo
                parsed = pd.to_datetime(ser, errors='coerce', dayfirst=True)
                info['fechas_parseables'] = int(parsed.notna().sum())
                info['muestras'] = list(ser.dropna().unique()[:10])
        chequeos_por_columna[col] = info

    filas_problemas = []
    for idx, fila in df.iterrows():
        lista_problemas = []
        if dup_mask.loc[idx]:
            lista_problemas.append('duplicado_exacto')
        for col in df.columns:
            val = fila[col]
            # heurísticas textuales
            if pd.api.types.is_string_dtype(type(val)) or isinstance(val, str) or (
                not pd.isna(val) and not pd.api.types.is_numeric_dtype(type(val)) and str(chequeos_por_columna[col].get('dtype','')).startswith('object')
            ):
                s = str(val)
                if s != s.strip():
                    lista_problemas.append(f'espacios_en_columna_{col}')
                if chequeos_por_columna[col].get('variantes_mayusculas'):
                    if s and s != s.lower() and s.lower() in [str(x).lower() for x in df[col].dropna().unique()]:
                        lista_problemas.append(f'inconsistencia_mayusculas_columna_{col}')
                if chequeos_por_columna[col].get('grupos_variantes_acentos') and chequeos_por_columna[col]['grupos_variantes_acentos'] > 0:
                    try:
                        un = sacar_acentos(s).lower()
                        group_vals = [x for x in chequeos_por_columna[col].get('muestras', []) if sacar_acentos(str(x)).lower() == un]
                        if group_vals and any(sacar_acentos(str(x)).lower() != sacar_acentos(s).lower() for x in group_vals):
                            lista_problemas.append(f'variantes_acentos_columna_{col}')
                    except Exception:
                        pass
                if es_valor_faltante(s):
                    lista_problemas.append(f'token_faltante_columna_{col}')
            else:
                # heurísticas numéricas
                try:
                    fval = float(val)
                    info_col = chequeos_por_columna[col]
                    limites = info_col.get('limites_iqr')
                    if limites and (fval < limites[0] or fval > limites[1]):
                        lista_problemas.append(f'outlier_iqr_columna_{col}')
                    if info_col.get('std') not in (None, 0):
                        mean = info_col.get('media')
                        std = info_col.get('std')
                        if std and abs((fval - mean) / std) > 3:
                            lista_problemas.append(f'outlier_z_columna_{col}')
                except Exception:
                    pass
        if lista_problemas:
            filas_problemas.append({
                'row_index': idx,
                'problemas': ';'.join(sorted(set(lista_problemas))),
                'muestra': json.dumps({str(c): str(fila[c]) for c in df.columns[:8]})
            })

    df_problemas = pd.DataFrame(filas_problemas)
    return resumen, chequeos_por_columna, df_problemas

print('Función detectar_problemas_en_dataframe cargada.')

Función detectar_problemas_en_dataframe cargada.


In [32]:
# Función auxiliar para guardar CSVs
def guardar_csv(df, ruta):
    """
    Guarda df en ruta (string o Path). Crea directorio padre si no existe.
    """
    ruta = Path(ruta)
    ruta.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(ruta, index=False, encoding='utf-8')
    return ruta

In [33]:
dic_frames = {
    ruta_ventas     : df_ventas,
    ruta_clientes   : df_clientes,
    ruta_marketing  : df_marketing
}

In [34]:
# ---------- Proceso principal para los 3 CSV ----------
#id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
reglas_marketing = {
                        'producto': ('lower', {'normalizar_acentos': True}),
                        'canal': ('lower', {'normalizar_acentos': True}),
                        'costo': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'fecha_inicio': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                        'fecha_fin': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']})
}
# id_venta,producto,precio,cantidad,fecha_venta,categoria
reglas_ventas = {
                        'producto': ('lower', {'normalizar_acentos': True}),
                        'precio': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'cantidad': ('numeric', {'remove_thousands': True, 'as_int': False}),
                        'fecha_venta': ('date', {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                        'categoria': ('lower', {'normalizar_acentos': True})
}
#id_cliente,nombre,edad,ciudad,ingresos
reglas_clientes = {
                        'nombre': ('title', {'normalizar_acentos': True}),
                        'edad': ('numeric', {'remove_thousands': True, 'as_int': True}),
                        'ciudad': ('title', {'normalizar_acentos': True}),
                        'ingresos': ('numeric', {'remove_thousands': True, 'as_int': False})
}

reglas_por_archivo = {
    'marketing.csv': reglas_marketing,
    'ventas.csv'   : reglas_ventas,
    'clientes.csv' : reglas_clientes
}
reportes_creados = []
zip_path = carpeta_reportes.parent / 'reports_dataset_tpi_v2.zip'


In [35]:
# ---------- Bucle/menu principal (usa dic_frames) ----------
def menu_procesar_diccionario(dic_frames, reglas_por_archivo):
    """
    Recorre dic_frames: clave = nombre_archivo (ej. 'marketing.csv'), valor = DataFrame.
    Ejecuta: detectar_problemas_en_dataframe antes, limpiar_dataframe, detectar_problemas_en_dataframe despues,
    imprime resúmenes y guarda cleaned en carpeta_limpios con sufijo ' limpio.csv'.
    También sobreescribe variables en RAM (df_marketing, df_ventas, df_clientes) si se encuentran en el nombre.
    """
    # trabajamos sobre una copia para evitar modificar dict original por error
    for nombre_archivo, df_actual in dic_frames.items():
        resumen_antes, chequeos_antes, problemas_antes = detectar_problemas_en_dataframe(df_actual)
        print(f"--- RESUMEN ANTES: {nombre_archivo} ---")
        print(resumen_antes)
        # Para no volcar objetos muy grandes, mostramos el head del DataFrame de problemas (si existe)
        print('Muestras de problemas antes (primeras 5 filas):')
        display(problemas_antes.head(5) if not problemas_antes.empty else 'No se detectaron filas con problemas.')

        reglas = reglas_por_archivo.get(nombre_archivo, {})
        df_limpio = limpiar_dataframe(df_actual, reglas_por_columna=reglas)

        resumen_despues, chequeos_despues, problemas_despues = detectar_problemas_en_dataframe(df_limpio)
        print(f"n--- RESUMEN DESPUÉS: {nombre_archivo} ---")
        print(resumen_despues)
        print('Muestras de problemas después (primeras 5 filas):')
        display(problemas_despues.head(5) if not problemas_despues.empty else 'No se detectaron filas con problemas tras la limpieza.')

        # mostrar separadores y tipo-nombre
        print('-'*100)
        print(f'nombre_archivo = {nombre_archivo}')
        print(f'type(nombre_archivo) = {type(nombre_archivo)}')
        print('-'*100)

        ruta_nombre_limpio = nombre_archivo[:-4] + '_limpio.csv' if nombre_archivo.lower().endswith('.csv') else nombre_archivo + ' limpio.csv'
        print(f'ruta_nombre_limpio = {ruta_nombre_limpio}')

        # guardar cleaned
        ruta_guardado = carpeta_limpios / ruta_nombre_limpio
        guardar_csv(df_limpio, ruta_guardado)
        print(f'Guardado cleaned en: {ruta_guardado}')

        # sobreescribir en RAM según el nombre
        # (nota: usar globals() para actualizar variables en el entorno global del notebook)
        if 'marketing' in nombre_archivo.lower():
            globals()['df_marketing'] = df_limpio
        elif 'ventas' in nombre_archivo.lower():
            globals()['df_ventas'] = df_limpio
        elif 'clientes' in nombre_archivo.lower():
            globals()['df_clientes'] = df_limpio

    print('Proceso completo del diccionario de DataFrames.')

In [36]:
menu_procesar_diccionario(dic_frames, reglas_por_archivo)

AttributeError: 'str' object has no attribute 'dropna'